# Reconstruyendo la configuración inicial del primer notebook

Se cargan las librerías iniciales.

In [ ]:
import torch
import os
from diffusers import UNet2DModel, DDPMPipeline, DDPMScheduler
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from datasets import load_dataset
from torchvision import transforms
from accelerate import Accelerator
from torch.utils.data import DataLoader
from tqdm.auto import tqdm  # Línea de progreso para el entrenamiento

Se replica la configuración del notebook anterior, añadiéndole más épocas.

In [ ]:
from dataclasses import dataclass

@dataclass
class TrainingConfig:
    image_size = 128
    train_batch_size = 10
    eval_batch_size = 16
    num_epochs = 50 + 150 # Se le añaden otras 150 épocas para que quede mejor entrenado, espero
    gradient_accumulation_steps = 1
    learning_rate = 1e-4
    lr_warmup_steps = 500
    save_image_epochs = 10
    save_model_epochs = 30
    mixed_precision = "fp16"
    seed = 42
    output_dir = "ddpm-cars-128"

    push_to_hub = False

config = TrainingConfig()

## Cargando los estados del notebook anterior.

In [ ]:
from datasets import load_dataset

config.dataset_name = "tanganke/stanford_cars"
dataset = load_dataset(config.dataset_name, split="train")

In [ ]:
from torchvision import transforms

# Definir las transformaciones que se aplicarán a cada imagen en el dataset
preprocess = transforms.Compose([
    transforms.Resize((config.image_size, config.image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),  # Normaliza imágenes RGB
])

In [ ]:
def transform(examples):
    images = [preprocess(image.convert("RGB")) for image in examples["image"]]
    return {"images": images}

dataset.set_transform(transform)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=config.train_batch_size, shuffle=True)

## Cargando el modelo anterior, junto al scheduler y al optimizador

In [ ]:
# Cargar el modelo guardado previamente desde el directorio de salida
model = UNet2DModel.from_pretrained(os.path.join(config.output_dir, "unet"))

# Inicializar el optimizer y cargar su estado guardado
optimizer = AdamW(model.parameters(), lr=config.learning_rate)
optimizer_state_path = os.path.join(config.output_dir, "optimizer_state.pt")
optimizer.load_state_dict(torch.load(optimizer_state_path))

# Inicializar el scheduler y cargar su estado
lr_scheduler = CosineAnnealingLR(optimizer, T_max=config.num_epochs)
scheduler_state_path = os.path.join(config.output_dir, "scheduler_state.pt")
lr_scheduler.load_state_dict(torch.load(scheduler_state_path))

# Si también usaste un noise scheduler (como DDPM in Diffusers)
noise_scheduler = DDPMScheduler.from_pretrained(os.path.join(config.output_dir, "scheduler"))

/tmp/ipykernel_8637/589757197.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  optimizer.load_state_dict(torch.load(optimizer_state_path))
/tmp/ipykernel_8637/589757197.p

In [ ]:
from accelerate import Accelerator

# Inicializa el objeto Accelerator para gestionar entrenamiento en GPU en caso de estar disponible
accelerator = Accelerator(
    mixed_precision=config.mixed_precision,  # Asegura que sea igual a como lo configuraste antes
    gradient_accumulation_steps=config.gradient_accumulation_steps,  # Si estabas usando grad accumulation
    log_with="tensorboard",  # O Weights & Biases si lo preferías
    project_dir=os.path.join(config.output_dir, "logs"),  # Directorio para logs del entrenamiento
)

# Prepare el modelo, optimizer, dataloader, y scheduler para el entrenamiento (si tienes dataloader configurado)
model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
    model, optimizer, train_dataloader, lr_scheduler
)

/home/creep/workshop/diffusion-model/.env/lib/python3.12/site-packages/accelerate/accelerator.py:443: UserWarning: `log_with=tensorboard` was passed but no supported trackers are currently installed.
  warnings.warn(f"`log_with={log_with}` was passed but no supported trackers are currently installed.")


In [ ]:
from diffusers import DDPMPipeline
from diffusers.utils import make_image_grid
import os

def evaluate(config, epoch, pipeline):
    # Sample some images from random noise (this is the backward diffusion process).
    # The default pipeline output type is `List[PIL.Image]`
    images = pipeline(
        batch_size=config.eval_batch_size,
        generator=torch.Generator(device='cpu').manual_seed(config.seed), # Use a separate torch generator to avoid rewinding the random state of the main training loop
    ).images

    # Make a grid out of the images
    image_grid = make_image_grid(images, rows=4, cols=4)

    # Save the images
    test_dir = os.path.join(config.output_dir, "samples")
    os.makedirs(test_dir, exist_ok=True)
    image_grid.save(f"{test_dir}/{epoch:04d}.png")

## Volviendo a definir la función de entrenamiento, modificada para guardar snapshots cada 30 épocas

In [ ]:
from huggingface_hub import create_repo, upload_folder
from tqdm.auto import tqdm
from pathlib import Path
from torch.functional import F

def train_loop(config, model, noise_scheduler, optimizer, train_dataloader, lr_scheduler):
    if accelerator.is_main_process:
        if config.output_dir is not None:
            os.makedirs(config.output_dir, exist_ok=True)
        if config.push_to_hub:
            repo_id = create_repo(
                repo_id=config.hub_model_id or Path(config.output_dir).name, exist_ok=True
            ).repo_id
        accelerator.init_trackers("train_example")

    # Prepare everything
    # There is no specific order to remember, you just need to unpack the
    # objects in the same order you gave them to the prepare method.
    model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
        model, optimizer, train_dataloader, lr_scheduler
    )

    global_step = 0

    # Now you train the model
    for epoch in range(50, config.num_epochs):
        progress_bar = tqdm(total=len(train_dataloader), disable=not accelerator.is_local_main_process)
        progress_bar.set_description(f"Epoch {epoch}")

        for step, batch in enumerate(train_dataloader):
            clean_images = batch["images"]
            # Sample noise to add to the images
            noise = torch.randn(clean_images.shape, device=clean_images.device)
            bs = clean_images.shape[0]

            # Sample a random timestep for each image
            timesteps = torch.randint(
                0, noise_scheduler.config.num_train_timesteps, (bs,), device=clean_images.device,
                dtype=torch.int64
            )

            # Add noise to the clean images according to the noise magnitude at each timestep
            # (this is the forward diffusion process)
            noisy_images = noise_scheduler.add_noise(clean_images, noise, timesteps)

            with accelerator.accumulate(model):
                # Predict the noise residual
                noise_pred = model(noisy_images, timesteps, return_dict=False)[0]
                loss = F.mse_loss(noise_pred, noise)
                accelerator.backward(loss)

                if accelerator.sync_gradients:
                    accelerator.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

            progress_bar.update(1)
            logs = {"loss": loss.detach().item(), "lr": lr_scheduler.get_last_lr()[0], "step": global_step}
            progress_bar.set_postfix(**logs)
            accelerator.log(logs, step=global_step)
            global_step += 1

        # After each epoch you optionally sample some demo images with evaluate() and save the model
        if accelerator.is_main_process:
            pipeline = DDPMPipeline(unet=accelerator.unwrap_model(model), scheduler=noise_scheduler)

            if (epoch + 1) % config.save_image_epochs == 0 or epoch == config.num_epochs - 1:
                evaluate(config, epoch, pipeline)

            # Save the model in a unique directory for each epoch
            if (epoch + 1) % config.save_model_epochs == 0 or epoch == config.num_epochs - 1:
                epoch_output_dir = os.path.join(config.output_dir, f"epoch-{epoch + 1}")
                os.makedirs(epoch_output_dir, exist_ok=True)

                if config.push_to_hub:
                    upload_folder(
                        repo_id=repo_id,
                        folder_path=epoch_output_dir,
                        commit_message=f"Epoch {epoch}",
                        ignore_patterns=["step_*", "epoch_*"],
                    )
                else:
                    # Save model into the output directory specified for this epoch
                    pipeline.save_pretrained(epoch_output_dir)

                # Optionally, save the optimizer and scheduler states for each epoch
                torch.save(optimizer.state_dict(), os.path.join(epoch_output_dir, "optimizer_state.pt"))
                torch.save(lr_scheduler.state_dict(), os.path.join(epoch_output_dir, "scheduler_state.pt"))

In [ ]:
from accelerate import notebook_launcher

args = (config, model, noise_scheduler, optimizer, train_dataloader, lr_scheduler)

notebook_launcher(train_loop, args, num_processes=1)

Launching training on one GPU.


Epoch 50:   0%|          | 0/815 [00:00<?, ?it/s]

Epoch 199: 100%|██████████| 815/815 [04:55<00:00,  2.76it/s, loss=0.00728, lr=0, step=122249]


## Guardando el modelo, el scheduler y el optimizador.

In [ ]:
# Guardar modelo y estado de optimizer/scheduler según config.save_model_epochs como antes
model.save_pretrained(config.output_dir)
torch.save(optimizer.state_dict(), os.path.join(config.output_dir, "optimizer_state.pt"))
torch.save(lr_scheduler.state_dict(), os.path.join(config.output_dir, "scheduler_state.pt"))